,Id,Location,Name,FullName,Division,Conference
0,1,Buffalo,Bills,Buffalo Bills,AFC East,AFC
1,2,Miami,Dolphins,Miami Dolphins,AFC East,AFC
2,3,New England,Patriots,New England Patriots,AFC East,AFC
3,4,New York,Jets,New York Jets,AFC East,AFC
4,5,Kansas City,Chiefs,Kansas City Chiefs,AFC West,AFC
